<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#First-train" data-toc-modified-id="First-train-1">First train</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-1.3">Data</a></span><ul class="toc-item"><li><span><a href="#Code" data-toc-modified-id="Code-1.3.1">Code</a></span></li><li><span><a href="#Save-to-npz" data-toc-modified-id="Save-to-npz-1.3.2">Save to npz</a></span></li><li><span><a href="#Load-1000-matches-for-training" data-toc-modified-id="Load-1000-matches-for-training-1.3.3">Load 1000 matches for training</a></span></li></ul></li></ul></li></ul></div>

# First train

## Goal

The goal is to do the first training using the whole dataset. Later I will move this to script.

## Imports

In [1]:
# Use this to reload changes in python scripts
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
from kaggle_environments import make
import pandas as pd
from tqdm.notebook import tqdm

from luxai.utils import render_game_in_html, set_random_seed
from luxai.cunet import cunet_model, cunet_luxai_model, config
from luxai.input_features import make_input, expand_board_size_adding_zeros
from luxai.output_features import (
    create_actions_mask, create_output_features,
    UNIT_ACTIONS_MAP, CITY_ACTIONS_MAP)

2021-10-20 10:37:02.015446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-20 10:37:02.015466: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Loading environment football failed: No module named 'gfootball'


In [3]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (30, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Data

### Code

In [4]:
def load_match_from_json(filepath, player):
    with open(filepath, 'r') as f:
        match = json.load(f)
    
    board, features, unit_output, city_output = [], [], [], []
    for step in range(len(match) - 1):
        observation = match[step][0]['observation']
        if player:
            observation.update(match[step][player]['observation'])
        actions = match[step+1][player]['action'] # notice the step + 150
        if actions is None: # this can happen on timeout
            continue

        ret = make_input(observation)
        active_units_to_position, active_cities_to_position, units_to_position = ret[2:]
        if active_units_to_position or active_cities_to_position:
            board.append(ret[0])
            features.append(ret[1])
            unit_actions_mask = create_actions_mask(active_units_to_position, observation)
            city_actions_mask = create_actions_mask(active_cities_to_position, observation)
            unit_actions, city_actions = create_output_features(actions, units_to_position, observation)
            unit_output.append(np.concatenate([unit_actions, unit_actions_mask], axis=-1))
            city_output.append(np.concatenate([city_actions, city_actions_mask], axis=-1))

    board = np.array(board, dtype=np.float32)
    features = np.array(features, dtype=np.float32)
    unit_output = np.array(unit_output, dtype=np.float32)
    city_output = np.array(city_output, dtype=np.float32)
    #print('%i/%i' % (len(board), len(match) - 1)) #this shows how many states did not have available actions
    return dict(board=board, features=features, unit_output=unit_output, city_output=city_output)


def save_match_to_npz(filepath, match):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    np.savez_compressed(filepath, **match)
    
    
def load_match_from_npz(filepath):
    return dict(**np.load(filepath))

In [5]:
def load_best_n_matches(n_matches):
    matches = []
    for episode_id, player in tqdm(zip(df.EpisodeId[:n_matches], df.Index[:n_matches]), total=n_matches, desc='Loading matches'):
        npz_filepath = os.path.join(matches_cache_npz_dir, '%i_%i.npz' % (episode_id, player))

        if os.path.exists(npz_filepath):
            match = load_match_from_npz(npz_filepath)
        else:
            json_filepath = os.path.join(matches_json_dir, '%i.json' % episode_id)
            match = load_match_from_json(json_filepath, player)
            save_match_to_npz(npz_filepath, match)

        matches.append(match)
    return matches

In [6]:
def combine_data_for_training(matches):
    inputs = [
        np.concatenate([expand_board_size_adding_zeros(match['board']) for match in matches]),
        np.concatenate([match['features'] for match in matches]),
    ]
    print('Inputs shapes', [x.shape for x in inputs])
    outputs = [
        np.concatenate([expand_board_size_adding_zeros(match['unit_output']) for match in matches]),
        np.concatenate([expand_board_size_adding_zeros(match['city_output']) for match in matches]),
    ]
    print('Outputs shapes', [x.shape for x in outputs])
    return inputs, outputs

In [7]:
def load_train_and_test_data(n_matches, test_fraction):
    matches = load_best_n_matches(n_matches=n_matches)
    
    test_matches = [match for idx, match in enumerate(matches) if not idx%test_fraction]
    train_matches = [match for idx, match in enumerate(matches) if idx%test_fraction]
    
    print('Train matches: %i' % len(train_matches))
    train_data = combine_data_for_training(train_matches)
    print('Test matches: %i' % len(test_matches))
    test_data = combine_data_for_training(test_matches)
    
    return train_data, test_data

### Save to npz

In [8]:
matches_json_dir = '/home/gbarbadillo/luxai_ssd/matches_20211014/matches_json'
matches_cache_npz_dir = '/home/gbarbadillo/luxai_ssd/matches_20211014/matches_npz'

In [12]:
df = pd.read_csv('/mnt/hdd0/Kaggle/luxai/agent_selection.csv')
df.sort_values('FinalScore', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Id,EpisodeId,Index,Reward,State,SubmissionId,InitialConfidence,InitialScore,UpdatedConfidence,UpdatedScore,FinalScore
0,69945543,27424471,1,90009.0,2,23032370,36.889864,1560.389280,36.522500,1566.517103,1818.288755
1,69923394,27413397,0,650053.0,2,23032370,38.435234,1536.093066,38.058785,1541.342982,1818.288755
2,69849883,27376641,1,410038.0,2,23032370,87.761971,1111.109255,83.368953,1142.533822,1818.288755
3,69847811,27375605,1,150010.0,2,23032370,144.366002,871.859256,135.552630,896.074161,1818.288755
4,69847037,27375218,1,130011.0,2,23032370,185.000000,702.140727,170.000000,788.476153,1818.288755


Loading data from all the matches from json files will take around an hour, thus we are going to save the features to npz file so we can reduce that time down to 10 minutes.

However we could not load all the dataset into memory due to its size. I have computed that if we normalize the board size to 32x32 each match will take 56MB of RAM memory.
Thus loading 1000 files will take 56 GB of ram.

In [8]:
matches = []
for episode_id, player in tqdm(zip(df.EpisodeId.values, df.Index.values), total=len(df)):
    npz_filepath = os.path.join(matches_cache_npz_dir, '%i_%i.npz' % (episode_id, player))
    if os.path.exists(npz_filepath):
        continue
    else:
        json_filepath = os.path.join(matches_json_dir, '%i.json' % episode_id)
        match = load_match_from_json(json_filepath, player)
        save_match_to_npz(npz_filepath, match)

  0%|          | 0/12791 [00:00<?, ?it/s]

### Load 1000 matches for training

In [10]:
train_data, test_data = load_train_and_test_data(n_matches=500, test_fraction=20)

Loading matches:   0%|          | 0/500 [00:00<?, ?it/s]

Train matches: 475
Inputs shapes [(157817, 32, 32, 24), (157817, 1, 13)]
Outputs shapes [(157817, 32, 32, 11), (157817, 32, 32, 4)]
Test matches: 25
Inputs shapes [(8042, 32, 32, 24), (8042, 1, 13)]
Outputs shapes [(8042, 32, 32, 11), (8042, 32, 32, 4)]
